# Alice & Bob — BB84 QKD (Qiskit ≥ 1.1 compatible)
This notebook is updated for **Qiskit 1.x**: it replaces the removed
`qiskit.execute` helper, switches to `AerSimulator`, and explicitly transpiles
circuits before calling `backend.run()`.  Everything else works exactly as
before — read an image, establish a one‑time‑pad key via BB 84, encrypt it, and
have Bob decrypt it.

In [6]:

import secrets, pathlib, numpy as np, base64
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
backend = AerSimulator()            # new API (replaces qasm_simulator)
rng = secrets.SystemRandom()


In [7]:

# — user tweakables —
IMG_PATH        = pathlib.Path('download.png')
BLOCK_SIZE      = 5
ABORT_THRESHOLD = 0.11
SAMPLE_RATE     = 0.10


In [8]:

if not IMG_PATH.exists():
    raise FileNotFoundError(f'Missing {IMG_PATH}. Drop an image or edit IMG_PATH.')
img_bytes  = IMG_PATH.read_bytes()
n_img_bits = len(img_bytes)*8
print(f'Image size: {len(img_bytes):,} bytes ({n_img_bits:,} bits)')


Image size: 3,844 bytes (30,752 bits)


In [9]:

def random_bits(n):
    return (np.random.randint(2, size=n, dtype=np.uint8),
            np.random.randint(2, size=n, dtype=np.uint8))  # data, bases

def alice_circuit(bits, bases):
    qc = QuantumCircuit(len(bits), name='Alice')
    for i,(bit,basis) in enumerate(zip(bits,bases)):
        if bit: qc.x(i)
        if basis: qc.h(i)
    return qc

def bob_measure(alice_qc, bob_bases):
    n = len(bob_bases)
    qc_bob = QuantumCircuit(n, n, name='Bob')
    for i,basis in enumerate(bob_bases):
        if basis: qc_bob.h(i)
        qc_bob.measure(i,i)
    full = alice_qc.compose(qc_bob)
    compiled = transpile(full, backend)
    result   = backend.run(compiled, shots=1).result()
    bitstr   = next(iter(result.get_counts()))
    return np.fromiter(map(int, bitstr[::-1]), dtype=np.uint8)

def hamming(a,b): return np.count_nonzero(a!=b)


In [10]:

alice_key = []
bob_key   = []
rounds    = 0

while len(alice_key) < n_img_bits:
    rounds += 1
    data_A, bases_A = random_bits(BLOCK_SIZE)
    qc = alice_circuit(data_A, bases_A)

    bases_B = np.random.randint(2, size=BLOCK_SIZE, dtype=np.uint8)
    meas_B  = bob_measure(qc, bases_B)

    keep_mask = bases_A == bases_B
    sift_A    = data_A[keep_mask]
    sift_B    = meas_B[keep_mask]

    sample_mask = np.random.rand(len(sift_A)) < SAMPLE_RATE
    qber = hamming(sift_A[sample_mask], sift_B[sample_mask]) / max(1, sample_mask.sum())
    if qber > ABORT_THRESHOLD:
        raise RuntimeError(f'ABORT — QBER {qber:.2%} exceeds {ABORT_THRESHOLD:.0%}')

    sift_A = sift_A[~sample_mask]
    sift_B = sift_B[~sample_mask]
    match  = sift_A == sift_B
    alice_key.extend(sift_A[match].tolist())
    bob_key  .extend(sift_B[match].tolist())
    print(f'Round {rounds:>3}: {len(alice_key):,}/{n_img_bits} key bits \r', end='')

alice_key = np.array(alice_key[:n_img_bits], dtype=np.uint8)
bob_key   = np.array(bob_key  [:n_img_bits], dtype=np.uint8)
assert np.array_equal(alice_key, bob_key)
print(f'\nKey done: {len(alice_key):,} bits')


Round 13696: 30,753/30752 key bits 
Key done: 30,752 bits


In [11]:

def bits_to_bytes(bits):
    out = bytearray()
    for i in range(0,len(bits),8):
        byte = 0
        for j in range(8):
            if i+j < len(bits):
                byte = (byte << 1) | int(bits[i+j])
        out.append(byte)
    return bytes(out)

key_bytes = bits_to_bytes(alice_key)
cipher    = bytes(m ^ k for m,k in zip(img_bytes, key_bytes))

enc_path = IMG_PATH.with_suffix('.enc')
enc_path.write_bytes(cipher)

# Bob decrypts
dec_bytes = bytes(c ^ k for c,k in zip(cipher, key_bytes))
dec_path  = IMG_PATH.with_stem(IMG_PATH.stem + '_decrypted')
dec_path.write_bytes(dec_bytes)
assert dec_bytes == img_bytes
print('\u2705 Success — decrypted image identical.')


✅ Success — decrypted image identical.
